In [35]:
import pandas as pd
import os
import re
import emoji

raw_path = '../data/raw/'
processed_path = '../data/processed'

In [36]:
ishate_path = os.path.join(raw_path, 'ishate')

## Get ID range and reindex, filtering out non-relevant columns

In [37]:
def encode_labels(val):
    if val == 'HS':
        return 1
    elif val == 'Non-HS':
        return 0

    return pd.NA

In [38]:
# remove usernames, urls, tags, and extra whitespace. Cast everything to lowercase
def clean_text(text):
    cleaned = text
    cleaned = re.sub(r'#([^ ]*)', r'\1', cleaned)
    cleaned = re.sub(r'https.*[^ ]', 'URL', cleaned)
    cleaned = re.sub(r'http.*[^ ]', 'URL', cleaned)
    cleaned = re.sub(r'@([^ ]*)', '@USER', cleaned)
    cleaned = emoji.demojize(cleaned)
    cleaned = re.sub(r'(:.*?:)', r' \1 ', cleaned)
    cleaned = re.sub(' +', ' ', cleaned)
    cleaned = cleaned.lower()

    return cleaned

In [39]:
start, stop = 0, 0

splits = ['train', 'val', 'test']

for split in splits:

    df_path = os.path.join(ishate_path, f'ishate_{split}.parquet.gzip')
    df = pd.read_parquet(df_path)
    stop += df.shape[0]
    df['id'] = range(start, stop)
    df['label_name'] = df['hateful_layer']
    df['implicit_label'] = df['implicit_layer']
    df['label'] = df['label_name'].apply(encode_labels)
    df['cleaned_text'] = df['text'].apply(clean_text)
    start += df.shape[0]
    df = df[['id', 'text', 'cleaned_text', 'implicit_label', 'label_name', 'label']]
    # df.insert(0, 'id', df.index)
    
    save_path = f'ishate_{split}.csv'
    print(save_path)

    df.to_csv(os.path.join(processed_path, save_path), index=False)
    print(df['id'].describe())

ishate_train.csv
count    20381.000000
mean     10190.000000
std       5883.632254
min          0.000000
25%       5095.000000
50%      10190.000000
75%      15285.000000
max      20380.000000
Name: id, dtype: float64
ishate_val.csv
count     4367.000000
mean     22564.000000
std       1260.788642
min      20381.000000
25%      21472.500000
50%      22564.000000
75%      23655.500000
max      24747.000000
Name: id, dtype: float64
ishate_test.csv
count     4368.000000
mean     26931.500000
std       1261.077317
min      24748.000000
25%      25839.750000
50%      26931.500000
75%      28023.250000
max      29115.000000
Name: id, dtype: float64
